# Scraping Wikipedia

[View on Github](https://github.com/villagecomputing/superpipe/tree/main/docs/examples/web_scraping/web_scraping.ipynb)

We'll use Superpipe to build a pipeline that receives a famous person's name and figures out their birthday, whether they're still alive and if not, their cause of death.

This pipeline will work in 4 steps -

1. Do a google search with the person's name
2. Use an LLM to fetch the URL of their wikipedia page from the search results
3. Fetch the contents of the wikipedia page and convert them to markdown
4. Use an LLM to extract the birthdate and living or dead from the wikipedia contents

We'll build the pipeline, evaluate it on some data, and optimize it to maximize accuracy while reducing cost and latency.

## Step 1: Building the pipeline

In [1]:
from superpipe.steps import LLMStructuredStep, CustomStep, SERPEnrichmentStep, LLMStructuredCompositeStep
from superpipe import models
from pydantic import BaseModel, Field

# Step 1: use Superpipe's built-in SERP enrichment step to search for the persons wikipedia page
# Include a unique "name" for the step that will used to reference this step's output in future steps

search_step = SERPEnrichmentStep(
  prompt= lambda row: f"{row['name']} wikipedia",
  name="search"
)

# Step 2: Use an LLM to extract the wikipedia URL from the search results
# First, define a Pydantic model that specifies the structured output we want from the LLM

class ParseSearchResult(BaseModel):
  wikipedia_url: str = Field(description="The URL of the Wikipedia page for the person")

# Then we use the built-in LLMStructuredStep and specify a model and a prompt
# The prompt is a function that has access to all the fields in the input as well as the outputs of previous steps

parse_search_step = LLMStructuredStep(
  model=models.gpt35,
  prompt= lambda row: f"Extract the Wikipedia URL for {row['name']} from the following search results: \n\n {row['search']}",
  out_schema=ParseSearchResult,
  name="parse_search"
)

In [2]:
db_model = "databricks-dbrx-instruct"
claude3_opus = "anthropic/claude-3-opus:beta"
claude3_sonnet = "anthropic/claude-3-sonnet:beta"
claude3_haiku = "anthropic/claude-3-haiku:beta"

# dbrx_url = "https://dbc-32549b5c-f244.cloud.databricks.com/serving-endpoints"
# openrouter_url = "https://openrouter.ai/api/v1"

# set_client_for_model(db_model, os.getenv("DATABRICKS_TOKEN"), dbrx_url)
# set_client_for_model(claude3_opus, os.getenv("OPENROUTER_API_KEY"), openrouter_url)
# set_client_for_model(claude3_sonnet, os.getenv("OPENROUTER_API_KEY"), openrouter_url)
# set_client_for_model(claude3_haiku, os.getenv("OPENROUTER_API_KEY"), openrouter_url)

In [3]:
from superpipe.pipeline import Pipeline
import requests
import html2text
import json

h = html2text.HTML2Text()
h.ignore_links = True

# Step 3: we create a CustomStep that can execute any arbitrary function (transform)
# The function fetches the contents of the wikipedia url and converts them to markdown

fetch_wikipedia_step = CustomStep(
  transform=lambda row: h.handle(requests.get(row['wikipedia_url']).text),
  name="wikipedia"
)

# Step 4: we extract the date of birth, living/dead status and cause of death from the wikipedia contents

class ExtractedData(BaseModel):
    date_of_birth: str = Field(description="The date of birth of the person in the format YYYY-MM-DD")
    alive: bool = Field(description="Whether the person is still alive")
    cause_of_death: str = Field(description="The cause of death of the person. If the person is alive, return 'N/A'")

extract_step = LLMStructuredCompositeStep(
  model=claude3_opus,
  prompt= lambda row: f"""Extract the date of birth for {row['name']}, whether they're still alive \
  and if not, their cause of death from the following Wikipedia content: \n\n {row['wikipedia']}""",
  out_schema=ExtractedData,
  name="extract_data"
)

# Finally we define and run the pipeline

pipeline = Pipeline([
  search_step,
  parse_search_step,
  fetch_wikipedia_step,
  extract_step
])

input = {"name": "Jean-Paul Sartre"}
output = pipeline.run(input)
print(json.dumps(output, indent=2))

{
  "name": "Jean-Paul Sartre",
  "search": "{\"searchParameters\":{\"q\":\"Jean-Paul Sartre wikipedia\",\"type\":\"search\",\"engine\":\"google\"},\"organic\":[{\"title\":\"Jean-Paul Sartre - Wikipedia\",\"link\":\"https://en.wikipedia.org/wiki/Jean-Paul_Sartre\",\"snippet\":\"Sartre was one of the key figures in the philosophy of existentialism (and phenomenology). His work has influenced sociology, critical theory, post-colonial ...\",\"sitelinks\":[{\"title\":\"Nausea (novel)\",\"link\":\"https://en.wikipedia.org/wiki/Nausea_(novel)\"},{\"title\":\"The Wall (Sartre short story...\",\"link\":\"https://en.wikipedia.org/wiki/The_Wall_(Sartre_short_story_collection)\"},{\"title\":\"Simone de Beauvoir\",\"link\":\"https://en.wikipedia.org/wiki/Simone_de_Beauvoir\"},{\"title\":\"Agr\u00e9gation\",\"link\":\"https://en.wikipedia.org/wiki/Agr%C3%A9gation\"}],\"position\":1},{\"title\":\"Jean-Paul Sartre - Simple English Wikipedia, the free encyclopedia\",\"link\":\"https://simple.wikipedia

In [4]:
input

{'name': 'Jean-Paul Sartre',
 'search': '{"searchParameters":{"q":"Jean-Paul Sartre wikipedia","type":"search","engine":"google"},"organic":[{"title":"Jean-Paul Sartre - Wikipedia","link":"https://en.wikipedia.org/wiki/Jean-Paul_Sartre","snippet":"Sartre was one of the key figures in the philosophy of existentialism (and phenomenology). His work has influenced sociology, critical theory, post-colonial ...","sitelinks":[{"title":"Nausea (novel)","link":"https://en.wikipedia.org/wiki/Nausea_(novel)"},{"title":"The Wall (Sartre short story...","link":"https://en.wikipedia.org/wiki/The_Wall_(Sartre_short_story_collection)"},{"title":"Simone de Beauvoir","link":"https://en.wikipedia.org/wiki/Simone_de_Beauvoir"},{"title":"Agrégation","link":"https://en.wikipedia.org/wiki/Agr%C3%A9gation"}],"position":1},{"title":"Jean-Paul Sartre - Simple English Wikipedia, the free encyclopedia","link":"https://simple.wikipedia.org/wiki/Jean-Paul_Sartre","snippet":"Jean-Paul Charles Aymard Sartre (21 June 

## Step 2: Evaluating the pipeline

Now, we'll evaluate the pipeline on a dataset. Think of this as unit tests for your code. You wouldn't ship code to production without testing it, you shouldn't ship LLM pipelines to production without evaluating them.

To do this, we need:

1. **A dataset with labels** - In this case we need a list of famous people and the true date of birth, living status and cause of death of each person
2. **Evaluation function** - a function that defines what "correct" is. We'll use simple comparison for date of birth and living status, and an LLM call to evaluate the correctness of cause of death.

In [ ]:
import pandas as pd

data = [
  ("Ruth Bader Ginsburg", "1933-03-15", False, "Pancreatic cancer"),
  ("Bill Gates", "1955-10-28", True, "N/A"),
  ("Steph Curry", "1988-03-14", True, "N/A"),
  ("Scott Belsky", "1980-04-18", True, "N/A"),
  ("Steve Jobs", "1955-02-24", False, "Pancreatic tumor/cancer"),
  ("Paris Hilton", "1981-02-17", True, "N/A"),
  ("Kurt Vonnegut", "1922-11-11", False, "Brain injuries"),
  ("Snoop Dogg", "1971-10-20", True, "N/A"),
  ("Kobe Bryant", "1978-08-23", False, "Helicopter crash"),
  ("Aaron Swartz", "1986-11-08", False, "Suicide")
]
df = pd.DataFrame([{"name": d[0], "dob_label": d[1], "alive_label": d[2], "cause_label": d[3]} for d in data])

class EvalResult(BaseModel):
  result: bool = Field(description="Is the answer correct or not?")

cause_evaluator = LLMStructuredStep(
  model=models.gpt4,
  prompt=lambda row: f"This is the correct cause of death: {row['cause_label']}. Is this provided cause of death accurate? The phrasing might be slightly different. Use your judgement: \n{row['cause_of_death']}",
  out_schema=EvalResult,
  name="cause_evaluator")

def eval_fn(row):
  score = 0
  if row['date_of_birth'] == row['dob_label']:
    score += 0.25
  if row['alive'] == row['alive_label']:
    score += 0.25
  if row['cause_label'] == "N/A":
    if row['cause_of_death'] == "N/A":
      score += 0.5
  elif cause_evaluator.run(row)['result']:
    score += 0.5  
  return score

pipeline.run(df)
print("Score: ", pipeline.evaluate(eval_fn))
df

Applying step parse_search:  20%|██        | 2/10 [00:01<00:04,  1.93it/s]

{'wikipedia_url': 'https://en.wikipedia.org/wiki/Ruth_Bader_Ginsburg'}


Applying step parse_search:  30%|███       | 3/10 [00:01<00:04,  1.50it/s]

{'wikipedia_url': 'https://en.wikipedia.org/wiki/Bill_Gates'}


Applying step parse_search:  40%|████      | 4/10 [00:02<00:04,  1.47it/s]

...

{'wikipedia_url': 'https://en.wikipedia.org/wiki/Kobe_Bryant'}


Applying step parse_search: 100%|██████████| 10/10 [00:08<00:00,  1.25it/s]


{'wikipedia_url': 'https://en.wikipedia.org/wiki/Aaron_Swartz'}


Applying step extract_data:  80%|████████  | 8/10 [00:11<00:02,  1.50s/it]


KeyboardInterrupt: 

In [ ]:
df['__parse_search__'].values

array([{'input_tokens': 1913, 'output_tokens': 23, 'input_cost': 0.0009565, 'output_cost': 3.45e-05, 'success': True, 'error': None, 'latency': 1.0774278750031954, 'content': {'wikipedia_url': 'https://en.wikipedia.org/wiki/Ruth_Bader_Ginsburg'}},
       {'input_tokens': 1723, 'output_tokens': 20, 'input_cost': 0.0008615, 'output_cost': 3e-05, 'success': True, 'error': None, 'latency': 1.3238214169978164, 'content': {'wikipedia_url': 'https://en.wikipedia.org/wiki/Bill_Gates'}},
       {'input_tokens': 1344, 'output_tokens': 20, 'input_cost': 0.000672, 'output_cost': 3e-05, 'success': True, 'error': None, 'latency': 0.5827157080057077, 'content': {'wikipedia_url': 'https://en.wikipedia.org/wiki/Stephen_Curry'}},
       {'input_tokens': 1130, 'output_tokens': 21, 'input_cost': 0.000565, 'output_cost': 3.15e-05, 'success': True, 'error': None, 'latency': 0.8819873329921393, 'content': {'wikipedia_url': 'https://en.wikipedia.org/wiki/Scott_Belsky'}},
       {'input_tokens': 1612, 'output_

## Step 3: Optimizing the pipeline

This pipeline has an accuracy score of 100%, but perhaps there's room for improvement on cost and speed. First let's view the cost and latency of each step to figure out which one is the bottleneck.

In [ ]:
for step in pipeline.steps:
  print(f"Step {step.name}:")
  print(f"- Latency: {step.statistics.total_latency}")
  print(f"- Cost: {step.statistics.input_cost + step.statistics.output_cost}")

Step search:
- Latency: 12.205872297286987
- Cost: 0.0
Step parse_search:
- Latency: 10.0048804559774
- Cost: 0.008355499999999998
Step wikipedia:
- Latency: 4.158370733261108
- Cost: 0.0
Step extract_data:
- Latency: 79.66739399997459
- Cost: 4.71622


Clearly the final step (`extract_data`) is the one responsible for the bulk of the cost and latency. This makes sense, because we're feeding in the entire wikipedia article to GPT-4, one of the most expensive models.

Let's find out if we can get away with a cheaper/faster model. Most models cannot handle the number of tokens needed to ingest a whole wikipedia article, so we'll turn to the two that can that are also cheaper than GPT4: Claude 3 Sonnet and Claude 3 Haiku.

In [ ]:
from superpipe.grid_search import GridSearch
# from superpipe.models import claude3_haiku, claude3_sonnet
from superpipe.steps import LLMStructuredCompositeStep

# we need to use LLMStructuredCompositeStep which uses GPT3.5 for structured JSON extraction
# because Claude does not support JSON mode or function calling out of the box
new_extract_step = LLMStructuredCompositeStep(
  model=models.claude3_haiku,
  prompt=extract_step.prompt,
  out_schema=ExtractedData,
  name="extract_data_new"
)

new_pipeline = Pipeline([
  search_step,
  parse_search_step,
  fetch_wikipedia_step,
  new_extract_step
], evaluation_fn=eval_fn)

param_grid = {
  new_extract_step.name:{
    "model": [claude3_haiku, claude3_sonnet]}
}
grid_search = GridSearch(new_pipeline, param_grid)
grid_search.run(df)

Applying step extract_data_new: 100%|██████████| 10/10 [00:14<00:00,  1.44s/it]
/Users/amandhesi/llm/superpipe/superpipe/util.py:44: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styler = styler.applymap(


,extract_data_new__model,score,input_cost,output_cost,total_latency,input_tokens,output_tokens,num_success,num_failure,index
0,anthropic/claude-3-haiku:beta,0.950000,0.007423,0.000312,37.120003,"defaultdict(, {'gpt-3.5-turbo-0125': 14847, 'anthropic/claude-3-haiku:beta': 0})","defaultdict(, {'gpt-3.5-turbo-0125': 208, 'anthropic/claude-3-haiku:beta': 0})",0,10,-954132478407494899
1,anthropic/claude-3-sonnet:beta,0.950000,0.007500,0.000312,35.606971,"defaultdict(, {'gpt-3.5-turbo-0125': 15000, 'anthropic/claude-3-sonnet:beta': 0})","defaultdict(, {'gpt-3.5-turbo-0125': 208, 'anthropic/claude-3-sonnet:beta': 0})",0,10,-6221134562231586183


Strangely, Claude 3 Haiku is both more accurate (100% v/s 45%) as well as cheaper and faster. This is suprising, but useful information that we wouldn't have found out unless we built and evaluated pipelines on _our specific data_ rather than benchmark data.

In [ ]:
best_params = grid_search.best_params
new_pipeline.update_params(best_params)
new_pipeline.run(df)
print("Score: ", new_pipeline.score)
for step in new_pipeline.steps:
  print(f"Step {step.name}:")
  print(f"- Latency: {step.statistics.total_latency}")
  print(f"- Cost: {step.statistics.input_cost + step.statistics.output_cost}")

Applying step extract_data_new: 100%|██████████| 10/10 [00:13<00:00,  1.36s/it]


Score:  0.95
Step search:
- Latency: 10.053870677947998
- Cost: 0.0
Step parse_search:
- Latency: 9.434886334027397
- Cost: 0.0074845
Step wikipedia:
- Latency: 3.5624284744262695
- Cost: 0.0
Step extract_data_new:
- Latency: 13.528477374988142
- Cost: 0.0


In [4]:
models_json = requests.get(f"https://openrouter.ai/api/v1/models").json()

In [7]:
models_json['data'][100]

{'id': 'anthropic/claude-1',
 'name': 'Anthropic: Claude v1',
 'description': "Anthropic's model for low-latency, high throughput text generation. Supports hundreds of pages of text.",
 'pricing': {'prompt': '0.000008',
  'completion': '0.000024',
  'image': '0',
  'request': '0'},
 'context_length': 100000,
 'architecture': {'modality': 'text',
  'tokenizer': 'Claude',
  'instruct_type': 'claude'},
 'top_provider': {'max_completion_tokens': 4096, 'is_moderated': True},
 'per_request_limits': None}